In [38]:
import pandas as pd

In [39]:
df = pd.read_csv('datasets/imdb_original/IMDB_Scraped.csv', dtype={'num_of_rating': str}, index_col=0)

# Converting non-USD to USD in `budget` column

In [40]:
budget = df['budget']

In [41]:
from forex_python.converter import CurrencyRates
c = CurrencyRates()

#  getting list of currencies
curr = budget.apply(lambda x: re.sub(r'\d+', '', x[:-12]).replace(',', '') if not pd.isna(x) else None).value_counts()

exchange = pd.DataFrame(columns=['currency', 'rate'])
cname = [
    'USD', 'EUR', 'INR', 'GBP', 'CAD', 'AUD', 'SEK ', 'NOK ', 'DEM ', 'RUR ', 'ZAR',
    'DKK', 'FRF ', 'FIM ', 'EGP ', 'HUF ', 'TRL', 'PLN', 'IRR ', 'CNY',
    'JPY', 'KRW', 'HKD', 'BDT ', 'ITL', 'CZK ', 'IDR', 'ESP ', 'MYR ', 'MXN',
    'NZD', 'NLG ', 'SGD ', 'CHF ', 'THB ', 'ARS ', 'PTE ', 'UAH ', 'ROL ',
    'ZAR ', 'NTD', 'ISK ', 'DOP ', 'PKR ', 'PHP', 'ILS', 'BGL ', 'HRK ', 'EEK',
    'AZM ', 'RON ', 'LVL ', 'BEF ', 'LKR ', 'LTL ', 'COP ', 'NPR ', 'AED ',
    'IEP ', 'CLP ', 'VND', 'YUM ', 'SIT ', 'GRD ', 'NGN ', 'VEB ', 'MVR ',
    'ATS ', 'MTL ', 'SAR ', 'PYG ', 'XAU ', 'MNT ', 'ALL ', 'IQD ', 'KWD ',
    'BND ', 'GEL ', 'JMD ', 'TMM '
    ]

In [42]:
#  getting currency rate for every curr in the curr list
rate = {}
for cu in list(curr.index):
    try:
        rate[cu] = c.convert(cname[list(curr.index).index(cu)].strip(), 'USD', 1)
    except:
        rate[cu] = None
rate

{'$': 1.0,
 '€': 1.0858,
 '₹': 0.012251829368057006,
 '£': 1.2287253304363572,
 'CA$': 0.7479506785148448,
 'A$': 0.7018745959922431,
 'SEK\xa0': 0.09782422631650074,
 'NOK\xa0': 0.10129392777513457,
 'DEM\xa0': None,
 'RUR\xa0': None,
 'R$': 0.0578516359683942,
 'DKK\xa0': 0.14594282181212115,
 'FRF\xa0': None,
 'FIM\xa0': None,
 'EGP\xa0': None,
 'HUF\xa0': 0.0027327410464852895,
 'TRL\xa0': 7.418037250844135e-07,
 'PLN\xa0': 0.23038404413324848,
 'IRR\xa0': None,
 'CN¥': 0.1474430352244643,
 '¥': 0.007675125468297166,
 '₩': 0.0008093321407274897,
 'HK$': 0.12772014021220035,
 'BDT\xa0': None,
 'ITL\xa0': None,
 'CZK\xa0': 0.045480438971265816,
 'IDR\xa0': 6.678126950836611e-05,
 'ESP\xa0': None,
 'MYR\xa0': 0.2337667929727868,
 'MX$': 0.053068625582225096,
 'NZ$': 0.6485098250014932,
 'NLG\xa0': None,
 'SGD\xa0': 0.7573411452884147,
 'CHF\xa0': 1.080075599323585,
 'THB\xa0': 0.03049229127467775,
 'ARS\xa0': None,
 'PTE\xa0': None,
 'UAH\xa0': None,
 'ROL\xa0': 3.3560921454343604e-05

In [43]:
#  converting 'budget' column
def change(x):
    if not pd.isna(x):
        try:
            amount = int(re.sub(r'[^0-9]', '', x))
            return amount*rate[re.sub(r'\d+', '', x[:-12]).replace(',', '')]
        except:
            return None
    else:
        return None
df['budget'] = budget.apply(change)

In [44]:
df['budget'].info()

<class 'pandas.core.series.Series'>
Int64Index: 113643 entries, 0 to 113642
Series name: budget
Non-Null Count  Dtype  
--------------  -----  
20581 non-null  float64
dtypes: float64(1)
memory usage: 1.7 MB


# Convert `num_of_rating` to int

In [45]:
def convert_num__of_rating(x):
    conv = {'M': 10**6, 'K': 10**3}
    if not pd.isna(x):
        try:
            return float(x[:-1])*conv[x[-1]]
        except KeyError:
            return float(x)
    else:
        return None
df['num_of_rating'] = df['num_of_rating'].apply(convert_num__of_rating)
df['num_of_rating']

0         2700000.0
1         2700000.0
2         2400000.0
3         2100000.0
4         2100000.0
            ...    
113638        181.0
113639        181.0
113640        181.0
113641        181.0
113642        181.0
Name: num_of_rating, Length: 113643, dtype: float64

# Adjust `budget` for inflation

In [47]:
import cpi
cpi.update()

In [78]:
import re
def extract_year_from_title(title):
    t = title.split(' ')
    if re.search(r'\(\d+\)', t[-1]):
        year = t[-1].strip('()')
        year = int(year)
        return year
    return None

In [82]:
#  adding year column to dataframe
df['year'] = df['title'].apply(extract_year_from_title)

In [155]:
df['inflated_budget'] = df.apply(lambda x: x.budget if x.year == 2023 else cpi.inflate(x.budget, int(x.year)) if x.year > 1912 else None, axis=1)